# **Decision Trees**
A short notebook to show the high variance of decision trees while using cost complexity pruning. We then compare different impurity functions.

## **Variance of decision trees**

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [ ]:
# 1. Generate simulated data
X, y = make_classification(n_samples=1000, n_features=20, n_informative=10, n_redundant=10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# 2. Train and prune decision tree using cross-validation
def prune_tree_with_ccp_alpha(X, y):
    dt = DecisionTreeClassifier(random_state=42)
    path = dt.cost_complexity_pruning_path(X, y)
    ccp_alphas = path.ccp_alphas
    cv_scores = []

    for ccp_alpha in ccp_alphas:
        dt = DecisionTreeClassifier(random_state=42, ccp_alpha=ccp_alpha)
        scores = cross_val_score(dt, X, y, cv=10, scoring='roc_auc')
        cv_scores.append(scores.mean())

    optimal_ccp_alpha = ccp_alphas[np.argmax(cv_scores)]
    pruned_tree = DecisionTreeClassifier(random_state=42, ccp_alpha=optimal_ccp_alpha)
    pruned_tree.fit(X, y)
    return pruned_tree, optimal_ccp_alpha

pruned_tree, optimal_ccp_alpha = prune_tree_with_ccp_alpha(X_train, y_train)
print(f"Optimal ccp_alpha: {optimal_ccp_alpha}")

In [ ]:
# 3. Show that changing slightly training data, the final decision trees varies a lot

# introduce a slight change in the training data
X_train_modified = X_train + np.random.normal(0, 0.1, X_train.shape)

# train the (pruned) tree on "perturbed data"
pruned_tree_modified, optimal_ccp_alpha_modified = prune_tree_with_ccp_alpha(X_train_modified, y_train)

# show the optimal ccp_alpha for the prunted tree on perturbed data
print(f"Optimal ccp_alpha on perturbed training data: {optimal_ccp_alpha_modified}")

In [ ]:
# Print number of terminal nodes and depth
print(f"Original Decision Tree - Number of terminal nodes: {pruned_tree.get_n_leaves()}, Depth: {pruned_tree.get_depth()}")
print(f"Modified Decision Tree - Number of terminal nodes: {pruned_tree_modified.get_n_leaves()}, Depth: {pruned_tree_modified.get_depth()}")

In [ ]:
# Plot original and modified trees
plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plot_tree(pruned_tree, filled=True)
plt.title("Original Decision Tree")

plt.subplot(1, 2, 2)
plot_tree(pruned_tree_modified, filled=True)
plt.title("Modified Decision Tree (Perturbed Data)")
plt.show()

In [ ]:
# 4. Compare results with respect to logistic regression
log_reg = LogisticRegression(random_state=42, max_iter=1000)
log_reg.fit(X_train, y_train)

# 5. Compute performance on test data using AUC
dt_pred_proba = pruned_tree.predict_proba(X_test)[:, 1]
dt_modified_pred_proba = pruned_tree_modified.predict_proba(X_test)[:, 1]
log_reg_pred_proba = log_reg.predict_proba(X_test)[:, 1]

dt_auc = roc_auc_score(y_test, dt_pred_proba)
dt_modified_auc = roc_auc_score(y_test, dt_modified_pred_proba)
log_reg_auc = roc_auc_score(y_test, log_reg_pred_proba)

# compare test performance
print(f"Decision Tree AUC: {dt_auc}")
print(f"Modified Decision Tree AUC: {dt_modified_auc}")
print(f"Logistic Regression AUC: {log_reg_auc}")
print()

# Compare ccp_alpha's
print(f"Original Decision Tree ccp_alpha: {optimal_ccp_alpha}")
print(f"Modified Decision Tree ccp_alpha: {optimal_ccp_alpha_modified}")
print(f"Difference in ccp_alpha: {abs(optimal_ccp_alpha - optimal_ccp_alpha_modified)}")

## **Impurity functions**
We check the impact of the impurity function on the structure and performance of the decision trees.

In [ ]:
# Function to prune decision tree using cross-validation with different impurity functions
def prune_tree_with_ccp_alpha(X, y, criterion):
    dt = DecisionTreeClassifier(criterion=criterion, random_state=42)
    path = dt.cost_complexity_pruning_path(X, y)
    ccp_alphas = path.ccp_alphas
    cv_scores = []

    for ccp_alpha in ccp_alphas:
        dt = DecisionTreeClassifier(criterion=criterion, random_state=42, ccp_alpha=ccp_alpha)
        scores = cross_val_score(dt, X, y, cv=10, scoring='roc_auc')
        cv_scores.append(scores.mean())

    optimal_ccp_alpha = ccp_alphas[np.argmax(cv_scores)]
    pruned_tree = DecisionTreeClassifier(criterion=criterion, random_state=42, ccp_alpha=optimal_ccp_alpha)
    pruned_tree.fit(X, y)
    return pruned_tree, optimal_ccp_alpha

In [ ]:
# Train and prune trees with different impurity functions
criteria = ['gini', 'entropy']
pruned_trees = {}
optimal_ccp_alphas = {}

for criterion in criteria:
    pruned_tree, optimal_ccp_alpha = prune_tree_with_ccp_alpha(X_train, y_train, criterion)
    pruned_trees[criterion] = pruned_tree
    optimal_ccp_alphas[criterion] = optimal_ccp_alpha
    print(f"Criterion: {criterion}, Optimal ccp_alpha: {optimal_ccp_alpha}")

In [ ]:
# Plot trees for each criterion
plt.figure(figsize=(20, 10))

for i, criterion in enumerate(criteria):
    plt.subplot(1, len(criteria), i + 1)
    plot_tree(pruned_trees[criterion], filled=True)
    plt.title(f"Decision Tree ({criterion})")

plt.show()

In [ ]:
# Print number of terminal nodes and depth for each criterion
for criterion in criteria:
    tree = pruned_trees[criterion]
    print(f"\nDecision Tree ({criterion})")
    print(f"Number of terminal nodes: {tree.get_n_leaves()}")
    print(f"Depth: {tree.get_depth()}")

In [ ]:
# Compare results with logistic regression
log_reg = LogisticRegression(random_state=42, max_iter=1000)
log_reg.fit(X_train, y_train)

# Compute performance on test data using AUC
auc_scores = {}
log_reg_pred_proba = log_reg.predict_proba(X_test)[:, 1]
log_reg_auc = roc_auc_score(y_test, log_reg_pred_proba)

for criterion in criteria:
    dt_pred_proba = pruned_trees[criterion].predict_proba(X_test)[:, 1]
    auc_scores[criterion] = roc_auc_score(y_test, dt_pred_proba)

# Print AUC scores
print("\nPerformance Comparison")
for criterion in criteria:
    print(f"Decision Tree ({criterion}) AUC: {auc_scores[criterion]}")
print(f"Logistic Regression AUC: {log_reg_auc}")

# Compare trees
print("\nTree Comparison")
for criterion in criteria:
    print(f"Decision Tree ({criterion}) ccp_alpha: {optimal_ccp_alphas[criterion]}")